In [ ]:
# Strip the header of raw files

from google.colab import files
import os
from datetime import datetime
from google.colab import drive
import shutil

def strip_header(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    modified_lines = lines[1:]  # Exclude the first line

    modified_content = ''.join(modified_lines)
    modified_file_path = os.path.splitext(file_path)[0] + '_stripped.txt'

    with open(modified_file_path, 'w') as modified_file:
        modified_file.write(modified_content)

    print(f'Stripped header and saved the modified file as: {modified_file_path}')

# Upload multiple files
uploaded_files = files.upload()

for file_name, file_content in uploaded_files.items():
    # Save the uploaded file
    with open(file_name, 'wb') as file:
        file.write(file_content)

    # Strip the header from the uploaded file
    strip_header(file_name)

    # Move the modified file to the specified Google Drive folder
    drive.mount('/content/drive')

    # Create a new folder with the current date in "My Drive/Colab Notebooks"
    base_folder = '/content/drive/My Drive/Colab Notebooks'
    current_date = datetime.now().strftime('%Y-%m-%d')
    folder_path = os.path.join(base_folder, current_date)
    os.makedirs(folder_path, exist_ok=True)

    # Move the modified file to the created folder
    modified_file_path = os.path.splitext(file_name)[0] + '_stripped.txt'
    new_file_path = os.path.join(folder_path, os.path.basename(modified_file_path))
    shutil.move(modified_file_path, new_file_path)

    print(f'Moved the modified file to: {new_file_path}')

In [ ]:
# Correct the time (starts from 0)

from google.colab import files
from datetime import date
import os

# Upload multiple TXT files
uploaded_files = files.upload()

# Output directory
output_dir = '/content/drive/MyDrive/Colab Notebooks/'

# Create the output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Create a subfolder with the current date
current_date = date.today().strftime("%Y-%m-%d")
output_dir = os.path.join(output_dir, current_date)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Process and save the files
for file_name, content in uploaded_files.items():
    # Get the file extension
    _, extension = os.path.splitext(file_name)
    
    # Check if the file is a TXT file
    if extension == '.txt':
        # Get the file content as string
        file_content = content.decode('utf-8')
        
        # Split the content into lines
        lines = file_content.strip().split('\n')

        # Determine the starting time (minimum value in the first column)
        starting_time = float(lines[0].split('\t')[0])

        # Initialize empty list for processed lines
        processed_lines = []

        # Process each line
        for line in lines:
            # Split the line into two values
            values = line.split('\t')
            time_value = (float(values[0]) - starting_time) / 60  # Normalize time to minutes starting from 0
            data_value = float(values[1])
            processed_lines.append('{}\t{}'.format(time_value, data_value))

        # Construct the processed content
        processed_content = '\n'.join(processed_lines)

        # Save the processed file
        output_file_name = os.path.join(output_dir, os.path.splitext(file_name)[0] + '_t-corrected' + extension)
        with open(output_file_name, 'w') as output_file:
            output_file.write(processed_content)
        
        print('Processed file saved:', output_file_name)

In [ ]:
# Split the staircase step

from google.colab import files
import os
from datetime import datetime
from google.colab import drive
import shutil

def process_file(file_path):
    # Move the selected file to the specified Google Drive folder
    drive.mount('/content/drive')

    # Create a new folder with the current date in "My Drive/Colab Notebooks"
    base_folder = '/content/drive/My Drive/Colab Notebooks'
    current_date = datetime.now().strftime('%Y-%m-%d')
    folder_path = os.path.join(base_folder, current_date)
    os.makedirs(folder_path, exist_ok=True)

    # Move the selected file to the created folder
    file_name = os.path.basename(file_path)
    new_file_path = os.path.join(folder_path, file_name)
    shutil.move(file_path, new_file_path)

    print(f'Moved the selected file to: {new_file_path}')

    # Split the data into time steps
    num_steps = int(input('Enter the number of time steps: '))

    # Read the file content
    with open(new_file_path, 'r') as file:
        lines = file.readlines()

    # Calculate the total duration
    last_line = lines[-1].split()
    total_duration = float(last_line[0]) if len(last_line) > 0 else 0.0

    # Set the time step duration
    time_step = 2  # minutes per time step

    # Calculate the step size
    step_size = time_step

    # Process and save the time steps
    for i in range(num_steps):
        start_time = i * step_size
        end_time = (i + 1) * step_size

        # Adjust the end time for the last step
        if i == num_steps - 1:
            end_time = total_duration

        # Filter the lines within the current time step
        time_step_lines = [line for line in lines if start_time <= float(line.split()[0]) <= end_time]

        # Create the processed file name
        processed_file_name = os.path.splitext(file_name)[0] + f'_step-{i+1}' + os.path.splitext(file_name)[1]
        processed_file_path = os.path.join(folder_path, processed_file_name)

        # Write the time step lines to the processed file
        with open(processed_file_path, 'w') as processed_file:
            processed_file.writelines(time_step_lines)

        print(f'Saved the processed file {processed_file_name}')

# Select the file using a browse window
uploaded_files = files.upload()

for file_name, file_content in uploaded_files.items():
    file_path = os.path.join(os.getcwd(), file_name)
    with open(file_path, 'wb') as file:
        file.write(file_content)

    process_file(file_path)

In [ ]:
# Correct the iRu

from google.colab import files
import os
from datetime import datetime

def process_file(file_name, file_content):
    # Read the file content
    lines = file_content.decode('utf-8').split('\n')

    # Get the fill-in values from the user
    reference_potential = float(input("Enter the reference potential (V): "))
    current = float(input("Enter the current (mA): "))
    resistance = float(input("Enter the resistance (Ω): "))

    # Process and update the file content
    processed_lines = []
    for line in lines:
        if line:
            time, value = line.strip().split('\t')
            new_value = float(value) - (current / 1000 * resistance) + reference_potential
            processed_line = '{}\t{}\n'.format(time, new_value)
            processed_lines.append(processed_line)

    # Get the current date
    current_date = datetime.now().strftime('%Y-%m-%d')

    # Create the output folder path
    base_folder = '/content/drive/My Drive/Colab Notebooks'
    folder_path = os.path.join(base_folder, current_date)
    os.makedirs(folder_path, exist_ok=True)

    # Create the processed file path
    processed_file_name = file_name.replace('.txt', '_iR-corrected.txt')
    processed_file_path = os.path.join(folder_path, processed_file_name)

    # Save the processed file
    with open(processed_file_path, 'w') as processed_file:
        processed_file.writelines(processed_lines)

    print('Processed file saved:', processed_file_path)

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Upload the file using a browse window
uploaded_files = files.upload()

# Process and update the file
for file_name, file_content in uploaded_files.items():
    process_file(file_name, file_content)

In [ ]:
# Merge the iRu corrected from multiple files into one

import os
import shutil
from datetime import datetime
from google.colab import drive, files

def merge_files(file_paths, output_file_path):
    with open(output_file_path, 'w') as output_file:
        for file_path in file_paths:
            with open(file_path, 'r') as input_file:
                output_file.write(input_file.read())

    print(f"Merged {len(file_paths)} files into: {output_file_path}")

# Select the files to merge using a browse window
uploaded_files = files.upload()

# Move the selected files to the specified Google Drive folder
drive.mount('/content/drive')

# Create a new folder with the current date in "My Drive/Colab Notebooks"
base_folder = '/content/drive/My Drive/Colab Notebooks'
current_date = datetime.now().strftime('%Y-%m-%d')
folder_path = os.path.join(base_folder, current_date)
os.makedirs(folder_path, exist_ok=True)

# Move the selected files to the created folder
file_paths = []
for file_name, file_content in uploaded_files.items():
    file_path = os.path.join(os.getcwd(), file_name)
    with open(file_path, 'wb') as file:
        file.write(file_content)

    new_file_path = os.path.join(folder_path, file_name)
    shutil.move(file_path, new_file_path)
    file_paths.append(new_file_path)

# Merge the files into one
output_file_name = file_paths[0].replace("_step-1_", "_").replace(".txt", "_merged.txt")
output_file_path = os.path.join(folder_path, output_file_name)
merge_files(file_paths, output_file_path)

In [ ]:
# Plot

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import os
import shutil
from datetime import datetime
from google.colab import drive, files

def process_file(file_path):
    # Read the input file
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Extract x and y values from the file
    x = []
    y = []
    for line in lines:
        values = line.strip().split('\t')  # Split using tabs
        x.append(float(values[0]))
        y.append(float(values[1]))

    return x, y

# Mount Google Drive
drive.mount('/content/drive')

# Select the files using a browse window
uploaded_files = files.upload()

fig, ax = plt.subplots()

for file_name, file_content in uploaded_files.items():
    file_path = os.path.join(os.getcwd(), file_name)
    with open(file_path, 'wb') as file:
        file.write(file_content)

    x, y = process_file(file_path)

    # Set the plot title
    file_base = os.path.splitext(file_name)[0]
    plot_title = f'{file_base}_plot'

    # Plot the data
    ax.plot(x, y, label=plot_title)

# Set the axis labels
ax.set_xlabel('$\mathit{t}$ / min')
ax.set_ylabel('$\mathit{E - iR_u}$ / V vs. RHE')

# Configure ticks
ax.minorticks_on()
ax.xaxis.set_minor_locator(ticker.AutoMinorLocator(4))
ax.yaxis.set_minor_locator(ticker.AutoMinorLocator(4))
ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
ax.yaxis.set_major_locator(ticker.MultipleLocator(0.1))

# Set x-axis scale from lowest to highest value
ax.set_xlim(min(x), max(x))

# Remove grid
ax.grid(False)

# Set the legend
ax.legend()

# Save the plot as PNG and SVG in the current date folder
current_date = datetime.now().strftime('%Y-%m-%d')
folder_path = os.path.join('/content/drive/My Drive/Colab Notebooks', current_date)
os.makedirs(folder_path, exist_ok=True)

plot_base_name = f'{current_date}_plot'
plot_file_path_png = os.path.join(folder_path, f'{plot_base_name}.png')
plot_file_path_svg = os.path.join(folder_path, f'{plot_base_name}.svg')

plt.savefig(plot_file_path_png, format='png')
plt.savefig(plot_file_path_svg, format='svg')

print(f'Plot saved as: {plot_file_path_png}')
print(f'Plot saved as: {plot_file_path_svg}')

# Move the plot files to the input file directory
input_file_dir = os.path.dirname(file_path)
shutil.copy(plot_file_path_png, os.path.join(input_file_dir, f'{plot_base_name}.png'))
shutil.copy(plot_file_path_svg, os.path.join(input_file_dir, f'{plot_base_name}.svg'))

print(f'Plot moved to: {os.path.join(input_file_dir, f"{plot_base_name}.png")}')
print(f'Plot moved to: {os.path.join(input_file_dir, f"{plot_base_name}.svg")}')